In [1]:
import pandas as pd

# Project imports
from self_harm_triage_notes.config import interim_data_dir, N_SPLITS
from self_harm_triage_notes.dataset_utils import print_stats

# Toolbox imports
from ml_health_toolbox.dev_utils import split_data, get_cv_strategy

___
# RMH 
### Load original data from 2012 to 2022

In [2]:
df = pd.read_parquet(interim_data_dir / "rmh_2012_2022_cleaned.parquet", engine="pyarrow")
print(df.shape)
df.head()

(771339, 14)


,uid,sex,age,arrival_method,arrival_date,year,triage_note,SH,SI,AOD_OD,audit_case,source_system,quarter,length
0,RMH-1,female,64.0,other,2012-01-08 00:35:00,2012,"SOB for 5/7, been to GP given prednisolone, co...",Negative,Negative,Negative,NaN,Symphony,2012Q1,140
1,RMH-2,male,31.0,other,2012-01-08 00:41:00,2012,"pt has lac down right forehead, to eyebrow, wi...",Negative,Negative,Negative,NaN,Symphony,2012Q1,107
2,RMH-3,male,19.0,road ambulance,2012-01-08 00:52:00,2012,"pt expect MBA, trapped for 45mins, #right femu...",Negative,Negative,Negative,NaN,Symphony,2012Q1,74
3,RMH-4,male,51.0,other,2012-01-08 01:11:00,2012,L) sided flank pain same as previous renal col...,Negative,Negative,Negative,NaN,Symphony,2012Q1,169
4,RMH-5,female,25.0,other,2012-01-08 01:23:00,2012,generalised abdo pain and associated headache ...,Negative,Negative,Negative,NaN,Symphony,2012Q1,196


In [3]:
df.year.value_counts().sort_index()

year
2012    59771
2013    61707
2014    65268
2015    65495
2016    71628
2017    75242
2018    77958
2019    82199
2020    86235
2021    83263
2022    42573
Name: count, dtype: int64

### Subset data: 2012-2017 for development and testing; 2018-2022

In [4]:
print("Number of presentations from 2018 onwards:", (df.year >= 2018).sum())
df[df.year >= 2018].to_parquet(interim_data_dir / "rmh_2018_2022_cleaned.parquet", engine="pyarrow")

Number of presentations from 2018 onwards: 372228


In [5]:
df = df[df.year <= 2017].copy()
print(df.shape)

(399111, 14)


### Create a train/test split

In [6]:
dev_data, test_data = split_data(df, df.SH, n_splits=5)

print("DEVELOPMENT SET")
print_stats(dev_data)
print()
print("TEST SET")
print_stats(test_data)

DEVELOPMENT SET
The dataset contains 319288 presentations.

SELF-HARM
Number of presentations:
SH
Negative    314962
Positive      4326
Name: count, dtype: int64

Proportion of presentations:
SH
Negative    98.64511
Positive     1.35489
Name: proportion, dtype: float64

________________________________________________________________________________
SUICIDAL IDEATION
Number of presentations:
SI
Negative    315457
Positive      3831
Name: count, dtype: int64

Proportion of presentations:
SI
Negative    98.800143
Positive     1.199857
Name: proportion, dtype: float64
________________________________________________________________________________
AOD overdose
Number of presentations:
AOD_OD
Negative    318634
Positive       654
Name: count, dtype: int64

Proportion of presentations:
AOD_OD
Negative    99.795169
Positive     0.204831
Name: proportion, dtype: float64


TEST SET
The dataset contains 79823 presentations.

SELF-HARM
Number of presentations:
SH
Negative    78741
Positive     1

In [7]:
# Initialise CV
X = dev_data 
y = dev_data.SH

# Initialise CV object
cv = get_cv_strategy(n_splits=N_SPLITS)

dev_data['val_fold'] = 0

i = 1
for train_idx, val_idx in cv.split(X, y, groups=None):
    print("%d reports allocated to validation fold #%d" % (len(val_idx), i))
    dev_data.loc[val_idx, 'val_fold'] = i
    i +=1

63858 reports allocated to validation fold #1
63858 reports allocated to validation fold #2
63858 reports allocated to validation fold #3
63857 reports allocated to validation fold #4
63857 reports allocated to validation fold #5


In [8]:
dev_data.to_parquet(interim_data_dir / "rmh_2012_2017_dev.parquet", engine="pyarrow")
test_data.to_parquet(interim_data_dir / "rmh_2012_2017_test.parquet", engine="pyarrow")